In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
class DecisionTreeRegressor:
    def fit(self, X, y, leaf_num):
        self.tree = node(X, y, np.array(np.arange(len(y))), 0, leaf_num)
        return self
    
    def predict(self, X):
        return self.tree.predict(X.values)
        

In [3]:
class node:
    
    def __init__(self, X, y, index, var_idx, leaf_num):
        self.X = X
        self.y = y
        self.var_idx = var_idx
        self.index = index
        self.leaf_num = leaf_num
        self.row_count = len(index)
        self.col_count = X.shape[1]
        self.val = np.mean(y[index])
        self.score = float('inf')
        self.find_varsplit()
        
    def print_var(self):
        print(self.var_idx)
    
    def find_varsplit(self):
        X = self.X.values[self.index, self.var_idx]
        
        for r in range(self.row_count):
            left_tree = X<=X[r]
            right_tree = X>X[r]
            if right_tree.sum() < self.leaf_num or left_tree.sum() < self.leaf_num: continue
            # 최소 leaf_num의 개수만큼은 있어야 한다. 적으면 밑의 내용 skip
            
            current_score = self.find_score(left_tree, right_tree)
           
            if current_score < self.score:
                self.score = current_score
                self.split_val = X[r]
                # split의 기준이 되는 값, 가장 score가 낮게 나온 경우의 값
    
        if self.is_leaf: return
        
        X = self.split_col
        left_tree = np.nonzero(X<=self.split_val)[0]
        right_tree = np.nonzero(X>self.split_val)[0]
        
        # self.print_var()
        
        if self.var_idx==7: 
            self.score = float('inf')
            return
        
        self.left_tree = node(self.X, self.y, self.index[left_tree], self.var_idx+1, self.leaf_num)
        self.right_tree = node(self.X, self.y, self.index[right_tree],  self.var_idx+1,self.leaf_num)
        
    
    def find_score(self, left_tree, right_tree):
        y = self.y[self.index]
        left_std = y[left_tree].std()
        right_std = y[right_tree].std()
        total_score =  left_std*left_tree.sum()+right_std*right_tree.sum()
        return total_score
    
    @property
    def split_col(self): 
        return self.X.values[self.index, self.var_idx]
    
    @property
    def is_leaf(self): 
        return self.score == float('inf')
    
  
    def predict(self, X):
        return np.array([self.predict_row(xi) for xi in X])
    
    def predict_row(self, xi):
        if self.is_leaf: return self.val
        # self.val = np.mean(y[idxs])
        # is_leaf의 경우에는 yi return
        node = self.left_tree if xi[self.var_idx] <=self.split_val else self.right_tree
        return node.predict_row(xi)

In [4]:
def normalize(data):
        # Normalize the data
        data = (data - data.mean())/data.std()
        return data

In [5]:
# r2_score function
def r2_score(y_true, y_pred):
    SS_total = 0
    SS_reg = 0
    
    for i in range(len(y_true)):
        x = (y_true[i] - y_pred[i])**2
        SS_reg = SS_reg+x
    
    y_true_mean = np.mean(y_true)
    for i in range(len(y_true)):
        x = (y_true[i]-y_true_mean)**2
        SS_total = SS_total+x
        
    
    print(1 - (SS_reg/SS_total))

In [6]:
def correlation_index(data, target):
    target = pd.DataFrame(target)
    corr_list = []
    for i in range(data.shape[1]):
        col_data = pd.DataFrame(data.iloc[:, i])
        corr = target.iloc[:,0].corr(col_data.iloc[:,0])
        corr_list.append(corr)
        
    
    corr_df = pd.DataFrame(columns = data.columns.values.tolist())
    corr_df.loc[len(corr_df)] = corr_list
   
    corr_df = pd.concat([corr_df, data], ignore_index=True)
    corr_df = corr_df.sort_values(by=0, axis=1, ascending=False)
    corr_df = corr_df.drop([0], axis=0)
    return corr_df

In [7]:
df_train = pd.read_csv('./test3_data/test3_data_train.csv')
df_train = normalize(df_train)

In [8]:
df_test = pd.read_csv('./test3_data/test3_data_test.csv')
df_test = normalize(df_test)

In [9]:
X_train = df_train.drop(['price_usd'], axis='columns')
y_train = df_train['price_usd']

In [10]:
X_test = df_test.drop(['price_usd'], axis='columns')
y_test = df_test['price_usd']

In [11]:
X_train = correlation_index(X_train, y_train)
cols = X_train.columns.tolist()
X_test = X_test[cols]

In [12]:
model = DecisionTreeRegressor()

In [13]:
model.fit(X_train, y_train, 15)

In [14]:
prediction = model.predict(X_test)

In [15]:
  pd.DataFrame({
    'y_true': y_test,
    'y_pred' : np.ravel(prediction)
})

,y_true,y_pred
0,-0.944395,-0.205254
1,0.685837,-0.205254
2,0.412423,0.015096
3,-0.038711,0.015096
4,0.494447,-0.379024
5,-0.189089,-0.205254
6,0.508117,-0.205254
7,0.002301,0.015096
8,0.685837,0.842148
9,0.214197,0.999581


In [16]:
r2_score(y_test, prediction)

0.26763491860914945
